In [1]:
import pandas as pd
import joblib
import shap

c:\Users\Utente\miniconda3\envs\cdk46i-metastatic-bc-ml\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
X_train = pd.read_excel("../Datasets/validation_dataset.xlsx")
X_train = X_train.drop(columns=['PD', 'PFS'])
X_train

,Age,Menopausal,Hormone_Resistance_Primary,Hormone_Resistance_Secondary,ER,PgR,HER2,Ki67,Histotype,M_Brain,M_Liver,M_Bone,M_Peritoneal,Bone_Only,Sinc
0,46.5,0.0,0,1,90.0,90.0,1,35.0,1.0,0,1,1,0,0,0.0
1,61.6,1.0,0,1,90.0,90.0,1,20.0,0.0,0,1,0,0,0,0.0
2,49.5,0.0,1,0,90.0,80.0,0,15.0,0.0,0,0,1,0,0,0.0
3,50.5,1.0,1,0,60.0,0.0,1,25.0,0.0,0,1,0,0,0,0.0
4,70.3,1.0,0,1,90.0,90.0,0,30.0,1.0,0,0,1,0,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,51.9,1.0,0,0,85.0,70.0,1,25.0,0.0,0,0,1,0,1,0.0
134,48.8,0.0,0,0,90.0,75.0,0,25.0,0.0,0,1,1,0,0,1.0
135,60.6,1.0,0,0,95.0,75.0,1,6.0,0.0,0,0,1,0,0,1.0
136,74.6,1.0,0,0,95.0,95.0,1,22.0,0.0,0,0,0,0,0,0.0


In [3]:
gbm_model = joblib.load('../Models/gbm_model.joblib')
gbm_model

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  SimpleImputer(strategy='median'),
                                                  ['Age', 'ER', 'PgR', 'Ki67']),
                                                 ('cat',
                                                  SimpleImputer(strategy='most_frequent'),
                                                  ['Menopausal',
                                                   'Hormone_Resistance_Primary',
                                                   'Hormone_Resistance_Secondary',
                                                   'HER2', 'Histotype',
                                                   'M_Brain', 'M_Liver',
                                                   'M_Bone', 'M_Peritoneal',
                                                   'Bone_Only', 'Sinc'])])),
                ('standardscaler', StandardScaler()),
                ('gradientboostingsurvivalanalysis',
                 GradientBoostingSurvivalAnalysis(learning_rate=1.0,
                                                  max_depth=1, n_estimators=16,
                                                  random_state=0))])

In [4]:
# separa preprocessing e modello
preprocess = gbm_model.named_steps["columntransformer"]
scaler = gbm_model.named_steps["standardscaler"]
model = gbm_model.named_steps["gradientboostingsurvivalanalysis"]

X_trans_train = preprocess.transform(X_train)
X_trans_train = scaler.transform(X_trans_train)

In [5]:
background = shap.sample(X_trans_train, 100, random_state=0)
joblib.dump(background, "shap_background_gbm.pkl")

['shap_background_gbm.pkl']